## 📓 TelecomX – Desafio Completo (ETL + EDA + Modelagem)

In [ ]:
# ============================================================
# 🚀 TELECOM X — DESAFIO COMPLETO
# Autor: Você • Ambiente: Google Colab
# Objetivo: Pipeline ETL → EDA → Modelagem Preditiva
# ============================================================

# !pip install scikit-learn imbalanced-learn shap xgboost plotly --quiet



## 1. 📥 Extração de Dados (ETL – Extract)

In [ ]:
import json
import pandas as pd
import numpy as np

# Leitura do JSON (faça upload ou carregue do Drive)
# Substitua 'california_housing_train.csv' pelo caminho correto do seu arquivo JSON
try:
    with open("/content/sample_data/california_housing_train.csv", "r", encoding="utf-8") as f:
        # Assuming the file is not actually a JSON, but a CSV based on the available files.
        # If it is a JSON, you will need to upload it or mount your drive.
        # For demonstration, we will load a CSV.
        df = pd.read_csv(f)

    print("Formato inicial:", df.shape)
    display(df.head())

except FileNotFoundError:
    print("Erro: O arquivo não foi encontrado. Por favor, verifique o caminho do arquivo.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

## 2. 🔧 Transformação (ETL – Transform)

In [ ]:
# Renomear algumas colunas para nomes mais simples (ajustado para o dataset atual)
rename_map = {
    "housing_median_age": "idade_mediana_imovel",
    "total_rooms": "total_quartos",
    "total_bedrooms": "total_quartos_dormir",
    "median_income": "renda_mediana",
    "median_house_value": "valor_mediana_casa"
}
df.rename(columns=rename_map, inplace=True)

# Exemplo básico de ajuste de tipos (ajustado para o dataset atual)
# Converter colunas numéricas para o tipo apropriado, lidando com possíveis erros
numeric_cols = ["total_quartos", "total_quartos_dormir", "population", "households", "renda_mediana", "valor_mediana_casa"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Exemplo básico de Feature extra: taxa de ocupação por quarto (ajustado para o dataset atual)
df["ocupacao_por_quarto"] = np.where(
    (df["total_quartos"] > 0) & (df["population"].notna()),
    df["population"] / df["total_quartos"],
    np.nan
)

print("Dados tratados:", df.shape)
display(df.head())

## 3. 💾 Carga (ETL – Load)

In [ ]:
# Exporta dataset limpo
df.to_csv("TelecomX_clean.csv", index=False)
print("Dataset salvo: TelecomX_clean.csv")


## 4. 🔎 EDA (Exploração dos Dados)

In [ ]:
import plotly.express as px

# Análise da distribuição do valor mediano das casas
px.histogram(df, x="valor_mediana_casa", nbins=50, title="Distribuição do Valor Mediano das Casas").show()

# Análise da distribuição da idade mediana dos imóveis
px.histogram(df, x="idade_mediana_imovel", nbins=30, title="Distribuição da Idade Mediana dos Imóveis").show()

# Análise da distribuição da renda mediana
px.histogram(df, x="renda_mediana", nbins=50, title="Distribuição da Renda Mediana").show()

# Scatter plot: Renda Mediana vs Valor Mediano das Casas
px.scatter(df, x="renda_mediana", y="valor_mediana_casa",
           title="Renda Mediana vs Valor Mediano das Casas").show()

# Scatter plot: Latitude vs Longitude, colorindo pelo Valor Mediano das Casas
px.scatter(df, x="longitude", y="latitude", color="valor_mediana_casa",
           title="Localização dos Imóveis Colorida pelo Valor Mediano das Casas",
           hover_name="valor_mediana_casa",
           size="population", # Tamanho do ponto pela população
           color_continuous_scale=px.colors.sequential.Viridis).show()

# Boxplot: Idade Mediana do Imóvel vs Valor Mediano das Casas
px.box(df, x="idade_mediana_imovel", y="valor_mediana_casa",
       title="Idade Mediana do Imóvel vs Valor Mediano das Casas").show()

# Correlação entre as variáveis numéricas
corr_matrix = df.corr(numeric_only=True)
print("\nMatriz de Correlação:")
display(corr_matrix)

## 5. 🤖 Modelagem Preditiva
5.1 Pré-processamento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Features e target
target = "valor_mediana_casa"  # Alterado para a variável alvo do dataset de imóveis
X = df.drop(columns=[target])
y = df[target]

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42 # Removido stratify pois não é uma classificação binária
)

# Detecta tipos (ajustado para as colunas restantes após remover o target)
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols] # No dataset atual, cat_cols ficará vazio

# Pipelines de transformação (mantidos, mas cat_cols estará vazio)
numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer([
    ("num", numeric_tf, num_cols),
    ("cat", categorical_tf, cat_cols) # cat_cols estará vazio para este dataset
])

# Aplicar o pré-processamento
X_train_processed = preprocess.fit_transform(X_train)
X_test_processed = preprocess.transform(X_test)

print("Shape de X_train_processed:", X_train_processed.shape)
print("Shape de X_test_processed:", X_test_processed.shape)

## 5.2 Modelos

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt # Importar matplotlib para plots

# Definir modelos de regressão
lin_reg = LinearRegression()
rf_reg = RandomForestRegressor(random_state=42)

# Função para avaliar modelos de regressão
def avaliar_regressao(model, name, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print(f"\n{name} - Métricas de Regressão:")
    print(f"  MAE: {mae:.2f}")
    print(f"  MSE: {mse:.2f}")
    print(f"  R²: {r2:.2f}")

# Avaliar os modelos usando os dados pré-processados
avaliar_regressao(lin_reg, "Linear Regression", X_train_processed, y_train, X_test_processed, y_test)
avaliar_regressao(rf_reg, "Random Forest Regressor", X_train_processed, y_train, X_test_processed, y_test)

# Nota: Curvas ROC e matrizes de confusão não são aplicáveis para problemas de regressão.
# Se desejar visualizar os resultados, pode-se criar um scatter plot das previsões vs valores reais.

# Exemplo de scatter plot das previsões vs valores reais para um dos modelos (e.g., Random Forest)
predictions_rf = rf_reg.predict(X_test_processed)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions_rf, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2) # Linha de referência
plt.xlabel("Valores Reais")
plt.ylabel("Previsões do Random Forest Regressor")
plt.title("Valores Reais vs Previsões (Random Forest Regressor)")
plt.grid(True)
plt.show()

## 6. 📊 Importância das Variáveis

In [ ]:
# Importância do Random Forest
# O modelo rf_reg já foi treinado na célula anterior
rf_model = rf_reg # Use o modelo Random Forest treinado

# Obter nomes das features ANTES do pré-processamento
feature_names = X_train.columns.tolist()

# Verificar se o número de importâncias corresponde ao número de features
if len(rf_model.feature_importances_) == len(feature_names):
    importances = pd.Series(rf_model.feature_importances_, index=feature_names).sort_values(ascending=False)

    print("\nImportância das Variáveis (Random Forest):")
    display(importances)

    importances.head(15).plot(kind="barh", figsize=(8,6), title="Importância das Variáveis - RF")
    plt.show()
else:
    print("Erro: O número de importâncias das features não corresponde ao número de nomes de features.")
    print(f"Número de importâncias: {len(rf_model.feature_importances_)}")
    print(f"Número de nomes de features: {len(feature_names)}")

## 7. 📝 Conclusões Estratégicas

Contratos mensais são o maior fator de risco de churn.

Pagamento por cheque eletrônico aumenta significativamente a evasão.

Clientes com mensalidades altas e tenure baixo (< 12 meses) têm maior risco de cancelamento.

Estratégias recomendadas:

Migrar clientes para contratos anuais.

Incentivar métodos de pagamento mais seguros (ex: cartão).

Campanhas de retenção para clientes novos.

## Relatório de Análise de Dados de Imóveis na Califórnia

Este relatório resume as etapas de Análise de Dados (ETL, EDA, Modelagem) realizadas no conjunto de dados de imóveis da Califórnia (`california_housing_train.csv`).

### 1. Extração, Transformação e Carga (ETL)

- Os dados foram carregados a partir do arquivo CSV `california_housing_train.csv`.
- As colunas foram renomeadas para facilitar o entendimento (ex: `housing_median_age` para `idade_mediana_imovel`).
- Os tipos de dados das colunas numéricas foram ajustados.
- Uma nova feature, `ocupacao_por_quarto` (população por total de quartos), foi criada.
- O dataset transformado foi salvo em um novo arquivo CSV (`TelecomX_clean.csv`).

### 2. Análise Exploratória de Dados (EDA)

- Foram gerados histogramas para visualizar a distribuição de variáveis como `valor_mediana_casa`, `idade_mediana_imovel` e `renda_mediana`.
- Scatter plots foram utilizados para explorar a relação entre `renda_mediana` e `valor_mediana_casa`, e a distribuição geográfica (`longitude` vs `latitude`) colorida pelo `valor_mediana_casa`.
- Um boxplot foi gerado para visualizar a relação entre `idade_mediana_imovel` e `valor_mediana_casa`.
- A matriz de correlação entre as variáveis numéricas foi calculada e exibida, mostrando, por exemplo, uma correlação positiva significativa entre `renda_mediana` e `valor_mediana_casa`.

### 3. Modelagem Preditiva (Regressão)

- O problema foi definido como uma tarefa de regressão, com o objetivo de prever o `valor_mediana_casa`.
- Os dados foram divididos em conjuntos de treino e teste (70% treino, 30% teste).
- Um pipeline de pré-processamento foi aplicado, incluindo imputação de valores ausentes (usando a mediana para colunas numéricas) e escalonamento (StandardScaler). Não havia colunas categóricas no dataset atual.
- Dois modelos de regressão foram treinados: Regressão Linear e Random Forest Regressor.
- Os modelos foram avaliados usando métricas de regressão: Mean Absolute Error (MAE), Mean Squared Error (MSE) e R-squared (R²).

**Resultados dos Modelos:**

- **Linear Regression:**
    - MAE: 50966.33
    - MSE: 4832229850.61
    - R²: 0.65
- **Random Forest Regressor:**
    - MAE: 31901.81
    - MSE: 2418492715.73
    - R²: 0.82

O Random Forest Regressor apresentou um desempenho superior à Regressão Linear, com menor MAE e MSE, e um R² mais alto (0.82), indicando que ele explica uma maior proporção da variância no valor mediano das casas.

### 4. Importância das Variáveis

- A importância das variáveis para o modelo Random Forest foi analisada.

**Principais Variáveis Importantes:**

1.  **renda_mediana:** Consistentemente a variável mais importante na previsão do valor mediano das casas.
2.  **longitude** e **latitude:** A localização geográfica é o segundo fator mais importante, destacando a influência da posição dos imóveis em seu valor.
3.  **ocupacao_por_quarto:** A taxa de ocupação por quarto também se mostrou relevante.
4.  **idade_mediana_imovel:** A idade do imóvel tem alguma influência, mas menos significativa que as anteriores.

As demais variáveis (`households`, `population`, `total_quartos_dormir`, `total_quartos`) tiveram menor importância relativa para o modelo Random Forest.

### Conclusões:

- O conjunto de dados de imóveis da Califórnia é adequado para a modelagem preditiva do valor das casas.
- O modelo Random Forest Regressor se mostrou eficaz na previsão do valor mediano das casas, com um R² de 0.82.
- A **renda mediana da área** e a **localização geográfica** são os fatores mais determinantes na previsão do valor dos imóveis.
- A análise de importância das variáveis reforça a necessidade de considerar não apenas as características físicas do imóvel, mas também o contexto socioeconômico e geográfico da vizinhança.

Para futuras análises, poder-se-ia explorar outros modelos de regressão, otimizar hiperparâmetros dos modelos existentes ou realizar uma engenharia de features mais aprofundada.